In [126]:
#Neural net with two hidden layer
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [127]:
#sigmoid
def sigmoid(x):
  return (1/(1+np.exp(-x)))

In [128]:
def cost(output,y):
  x=np.sum((-1*y)*np.log(output)-(1-y)*(np.log(1-output)))
  #print(x)
  return x

In [129]:
def prop(x,y,theta3,theta2,theta1,alpha):
  
  #Forward prop
  al1=np.dot(theta1.T,x)
  al1=sigmoid(al1)
  al2=np.dot(theta2.T,al1)
  al2=sigmoid(al2)
  output=np.dot(theta3.T,al2)
  output=sigmoid(output)
  ct=cost(output,y)

  #Backward prop
  delout=output-y
  del3=np.multiply((theta3)@delout,np.multiply(al2,(1-al2)))
  del2=np.multiply((theta2)@del3,np.multiply(al1,(1-al1)))
  
  theta3_grad=al2@delout.T
  theta2_grad=al1@del3.T
  theta1_grad=x@del2.T
 
  
  #update
  theta3=theta3-alpha*theta3_grad
  theta2=theta2-alpha*theta2_grad
  theta1=theta1-alpha*theta1_grad


  return [theta1,theta2,theta3,ct,output]
  

In [130]:

def MLP(x_train,y_train,x_test,check):
    theta1=np.random.rand(x_train.shape[1],10)
    theta2=np.random.rand(10,5)
    theta3=np.random.rand(5,1)

    alpha=0.05;
    x_train = x_train.values
    y =[0 for i in range(y_train.shape[0])]
    y_train = y_train.to_frame()
    for i in range(y_train.shape[0]):
        if y_train.iloc[i,0]==check:
            y[i]=1
        else:
            y[i]=0
    y = np.array(y)

  
    for j in range(1000):
      ct=0
      for i in range(x_train.shape[0]):
        theta1,theta2,theta3,cos,val=prop(x_train[i].reshape((x_train.shape[1],1)),y[i].reshape(1,1),theta3,theta2,theta1,alpha)
        ct+=cos


    print("Final cost :",ct/x_train.shape[0])
    
    x_test = x_test.values
    ct=0
    ans=[]
    for i in range(x_test.shape[0]):
          al1=np.dot(theta1.T,x_test[i].reshape((x_test.shape[1],1)))
          al1=sigmoid(al1)
          al2=np.dot(theta2.T,al1)
          al2=sigmoid(al2)
          output=np.dot(theta3.T,al2)
          output=sigmoid(output)
          if output[0]<=0.5:
              ans.append(0)
          else:
              ans.append(1)
            
    return ans

In [ ]:
data = pd.read_csv("SPECT.csv")
size = data.shape[0]
fold_size = int(size/10)
data = data.sample(frac=1)
accuracy=0
precision=0
recall=0
for i in range(10):
    test_x = data.iloc[i*fold_size:(i+1)*fold_size,1:] 
    test_y = data.iloc[i*fold_size:(i+1)*fold_size,0]
    if i!=0:
        train_x = data.iloc[:(i)*fold_size,1:] 
        train_x = train_x.append(data.iloc[(i+1)*fold_size:,1:])
        train_y = data.iloc[:(i)*fold_size,0] 
        train_y = train_y.append(data.iloc[(i+1)*fold_size:,0])
        
    else:
        train_x = data.iloc[(i+1)*fold_size:,1:]
        train_y = data.iloc[(i+1)*fold_size:,0]
        
    
    ans =  MLP(train_x,train_y,test_x,"Yes")
    acc=0
    tp=0
    tn=0
    fp=0
    fn=0
    for h in range(test_x.shape[0]):
        if(ans[h]<0):
            temp="No"
            if temp==test_y.iloc[h]:
                acc+=1
                tn+=1
            else:
                fn+=1
        else:
            temp="Yes"
            if temp==test_y.iloc[h]:
                acc+=1
                tp+=1
            else:
                fp+=1

    if i==0:
        print("1st Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
        
    elif i==1:
        print("2nd Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
        
    elif i==2:
        print("3rd Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
    else:
        print(i+1,"th Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
      
    print("Accuracy ",acc/test_x.shape[0])
    accuracy+=acc/test_x.shape[0]
    print("Precision ",tp/(tp+fp))
    precision +=tp/(tp+fp)
    print("Recall ",tp/(tp+fn))
    recall += tp/(tp+fn)
print("Average Accuracy =",accuracy/10)  
print("Average Precision =",precision/10)
print("Average Recall =",recall/10)  





Final cost : 0.12474588098015638
1st Fold -> Correctly classified 21 Incorrectly classified 5
Accuracy  0.8076923076923077
Precision  0.8076923076923077
Recall  1.0


In [ ]:
data = pd.read_csv("IRIS.csv")
size = data.shape[0]
fold_size = int(size/10)
data = data.sample(frac=1)
accuracy=0
precision=0
recall=0
for i in range(10):
    test_x = data.iloc[i*fold_size:(i+1)*fold_size,0:4]
    test_y = data.iloc[i*fold_size:(i+1)*fold_size,4]
    if i!=0:
        train_x = data.iloc[:(i)*fold_size,0:4] 
        train_x = train_x.append(data.iloc[(i+1)*fold_size:,0:4])
        train_y = data.iloc[:(i)*fold_size,4] 
        train_y = train_y.append(data.iloc[(i+1)*fold_size:,4])
        
    else:
        train_x = data.iloc[(i+1)*fold_size:,0:4]
        train_y = data.iloc[(i+1)*fold_size:,4]
        
    
    ans = MLP(train_x,train_y,test_x,'Iris-setosa')
    acc=0
    tp=0
    fp=0
    tn=0
    fn=0
    for h in range(test_x.shape[0]):
        if(ans[h]>0):
            temp="Iris-setosa"
            if temp==test_y.iloc[h]:
                acc+=1
                tp+=1
            else:
                fp+=1
        else:
            temp="Iris-versicolor"
            if temp==test_y.iloc[h]:
                acc+=1
                tn+=1
            else:
                fn+=1
        
    if i==0:
        print("1st Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
        
    elif i==1:
        print("2nd Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
        
    elif i==2:
        print("3rd Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
    else:
        print(i+1,"th Fold ->","Correctly classified",acc,"Incorrectly classified",test_x.shape[0]-acc)
      
    print("Accuracy ",acc/test_x.shape[0])
    accuracy+=acc/test_x.shape[0]
    print("Precision ",tp/(tp+fp))
    precision +=tp/(tp+fp)
    print("Recall ",tp/(tp+fn))
    recall += tp/(tp+fn)
print("Average Accuracy =",accuracy/10)  
print("Average Precision =",precision/10)
print("Average Recall =",recall/10)



